In [64]:
import pandas as pd
from sklearn import preprocessing
from transformers import AlbertTokenizer, AlbertForTokenClassification
from keras.preprocessing.sequence import pad_sequences
import torch

In [9]:
def sentence_label_getter(data_csv):
    df = pd.read_csv(data_csv, encoding='utf-8')
    sentences = df.groupby("sentence")["word"].apply(list).values
    labels = df.groupby("sentence")["label"].apply(list).values
    return sentences, labels

In [10]:
sentences, labels = sentence_label_getter('../input/train.csv')

In [11]:
sentences

array([list(['म्यूरियल', 'अब', 'बीस', 'साल', 'की', 'हो', 'गई', 'है']),
       list(['म्यूरियल', 'अब', 'बीस', 'साल', 'की', 'है']),
       list(['मैं', 'इस', 'दुनिया', 'में', 'शिक्षा', 'पर', 'बहुत', 'निराश', 'हूँ']),
       list(['वैसा', 'नही', 'होगा']),
       list(['मुझें', 'तुम्हारी', 'याद', 'आ', 'रही', 'है']),
       list(['तुम्हें', 'सोना', 'चाहिए']),
       list(['आपको', 'सोना', 'चाहिए']),
       list(['मुझे', 'जीव', 'विज्ञान', 'कभी', 'भी', 'पसंद', 'नहीं', 'था']),
       list(['मैं', 'नहीं', 'हूँ', 'तुम', 'हो'])], dtype=object)

In [12]:
labels

array([list(['none', 'none', 'none', 'none', 'none', 'none', 'none', 'viram']),
       list(['none', 'none', 'none', 'none', 'none', 'viram']),
       list(['none ', 'none', 'none', 'none', 'none', 'none', 'none', 'none', 'viram']),
       list(['none ', 'none', 'viram']),
       list(['none', 'none', 'none', 'none', 'none', 'viram']),
       list(['none ', 'none', 'viram']), list(['none ', 'none', 'viram']),
       list(['none ', 'none', 'none', 'none', 'none', 'none', 'none', 'viram']),
       list(['none ', 'none', 'comma', 'none', 'exclamation'])],
      dtype=object)

In [14]:
df = pd.read_csv('../input/train.csv')
tag_values = list(set(df["label"].values))

In [16]:
tag_values
tag_values.append("PAD")
tag2idx = {t: i for i, t in enumerate(tag_values)}

In [17]:
tag2idx

{'comma': 0, 'exclamation': 1, 'none': 2, 'viram': 3, 'none ': 4, 'PAD': 5}

In [20]:
tokenizer = AlbertTokenizer.from_pretrained('ai4bharat/indic-bert')

In [35]:
def tokenize_and_preserve_labels(sentence, text_labels):
    tokenized_sentence = []
    labels = []

    for word, label in zip(sentence, text_labels):

        # Tokenize the word and count # of subwords the word is broken into
        tokenized_word = tokenizer.tokenize(word)
        n_subwords = len(tokenized_word)

        # Add the tokenized word to the final tokenized word list
        tokenized_sentence.extend(tokenized_word)

        # Add the same label to the new list of labels `n_subwords` times
        labels.extend([label] * n_subwords)

    return tokenized_sentence, labels

In [36]:
tokenized_texts_and_labels = [
    tokenize_and_preserve_labels(sent, labs)
    for sent, labs in zip(sentences, labels)
]

In [37]:
tokenized_texts_and_labels

[(['▁म', 'यू', 'रियल', '▁अब', '▁बीस', '▁साल', '▁की', '▁हो', '▁गई', '▁है'],
  ['none',
   'none',
   'none',
   'none',
   'none',
   'none',
   'none',
   'none',
   'none',
   'none']),
 (['▁म', 'यू', 'रियल', '▁अब', '▁बीस', '▁साल', '▁की', '▁है'],
  ['none', 'none', 'none', 'none', 'none', 'none', 'none', 'none']),
 (['▁मैं',
   '▁इस',
   '▁दुनिया',
   '▁में',
   '▁शिक',
   'षा',
   '▁पर',
   '▁बहुत',
   '▁निराश',
   '▁हूँ'],
  ['none ',
   'none',
   'none',
   'none',
   'none',
   'none',
   'none',
   'none',
   'none',
   'none']),
 (['▁वैसा', '▁नही', '▁होगा'], ['none ', 'none', 'viram']),
 (['▁मुझे', 'ं', '▁तुम', 'हारी', '▁याद', '▁आ', '▁रही', '▁है'],
  ['none', 'none', 'none', 'none', 'none', 'none', 'none', 'none']),
 (['▁तुम', 'हें', '▁सोना', '▁चाहिए'], ['none ', 'none ', 'none ', 'none']),
 (['▁आपको', '▁सोना', '▁चाहिए'], ['none ', 'none', 'viram']),
 (['▁मुझे', '▁जीव', '▁विज', 'ञ', 'ान', '▁कभी', '▁भी', '▁पसंद', '▁नहीं', '▁था'],
  ['none ',
   'none',
   'none',
   'none',
   '

In [38]:
tokenized_texts = [token_label_pair[0] for token_label_pair in tokenized_texts_and_labels]
labels = [token_label_pair[1] for token_label_pair in tokenized_texts_and_labels]

In [39]:
tokenized_texts

[['▁म', 'यू', 'रियल', '▁अब', '▁बीस', '▁साल', '▁की', '▁हो', '▁गई', '▁है'],
 ['▁म', 'यू', 'रियल', '▁अब', '▁बीस', '▁साल', '▁की', '▁है'],
 ['▁मैं',
  '▁इस',
  '▁दुनिया',
  '▁में',
  '▁शिक',
  'षा',
  '▁पर',
  '▁बहुत',
  '▁निराश',
  '▁हूँ'],
 ['▁वैसा', '▁नही', '▁होगा'],
 ['▁मुझे', 'ं', '▁तुम', 'हारी', '▁याद', '▁आ', '▁रही', '▁है'],
 ['▁तुम', 'हें', '▁सोना', '▁चाहिए'],
 ['▁आपको', '▁सोना', '▁चाहिए'],
 ['▁मुझे', '▁जीव', '▁विज', 'ञ', 'ान', '▁कभी', '▁भी', '▁पसंद', '▁नहीं', '▁था'],
 ['▁मैं', '▁नहीं', '▁हूँ', '▁तुम', '▁हो']]

In [40]:
labels

[['none',
  'none',
  'none',
  'none',
  'none',
  'none',
  'none',
  'none',
  'none',
  'none'],
 ['none', 'none', 'none', 'none', 'none', 'none', 'none', 'none'],
 ['none ',
  'none',
  'none',
  'none',
  'none',
  'none',
  'none',
  'none',
  'none',
  'none'],
 ['none ', 'none', 'viram'],
 ['none', 'none', 'none', 'none', 'none', 'none', 'none', 'none'],
 ['none ', 'none ', 'none ', 'none'],
 ['none ', 'none', 'viram'],
 ['none ',
  'none',
  'none',
  'none',
  'none',
  'none',
  'none',
  'none',
  'none',
  'none'],
 ['none ', 'none', 'comma', 'none', 'exclamation']]

In [41]:
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=128, dtype="long", value=0.0,
                          truncating="post", padding="post")

In [44]:
tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels],
                     maxlen=128, value=tag2idx["PAD"], padding="post",
                     dtype="long", truncating="post")

In [45]:
attention_masks = [[float(i != 0.0) for i in ii] for ii in input_ids]

In [51]:
input_ids[0]

array([  2092,   7148, 103940,    311,  36287,    379,     17,     91,
          241,     16,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
      

In [53]:
attention_masks[0]

[1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [59]:
model = AlbertForTokenClassification.from_pretrained('ai4bharat/indic-bert',
                                                     num_labels=len(tag2idx),
                                                     output_attentions=False,
                                                     output_hidden_states=False)

Some weights of the model checkpoint at ai4bharat/indic-bert were not used when initializing AlbertForTokenClassification: ['predictions.bias', 'predictions.LayerNorm.weight', 'predictions.LayerNorm.bias', 'predictions.dense.weight', 'predictions.dense.bias', 'predictions.decoder.weight', 'predictions.decoder.bias', 'sop_classifier.classifier.weight', 'sop_classifier.classifier.bias']
- This IS expected if you are initializing AlbertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForTokenClassification were not initialized from the model checkpoint at ai4bharat/indic-bert and a

In [65]:
outputs = model(torch.tensor(input_ids[0]), token_type_ids=None,
                attention_mask=torch.tensor(attention_masks[0]),
                labels=torch.tensor(tags[0]))

IndexError: tuple index out of range

In [62]:
input_ids[0]

array([  2092,   7148, 103940,    311,  36287,    379,     17,     91,
          241,     16,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
      